In [1]:
## Kflod-5
import os
import torch
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensor, ToTensorV2
import torchvision.transforms as T

import time
import csv
import cv2
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
from PIL import Image
from args import build_opt
# from loaders import build_loader, build_loader_new
from tqdm.notebook import tqdm
# from torch.utils.tensorboard import SummaryWriter
opt = build_opt()

from utils import create_logger
import timm
## logger
logger = create_logger('../logging')
##
from sklearn.model_selection import KFold
import csv
import pandas as pd
from itertools import islice
import wandb

...........................................................................
...param: traindir: ../train/labeled_data/
...param: train_metadir: ../
...param: train_metafile: train_label.csv
...param: testdir: ../test
...param: test_metadir: None
...param: test_metafile: None
...param: epochs: 10
...param: num_workers: 4
...param: batch_size: 64
...param: lr: 0.1
...param: weight_decay: 1e-05
...param: num_classes: 2
...param: step_size: 5
...param: gamma: 0.9
...param: ckpt_name: None
...param: submission_name: submission.csv
...param: model_name: None
...param: kfold: 5
...param: device: cuda
...param: resultdir: ./result
...........................................................................


In [2]:
## 重置opt
opt.epochs = 5
opt.lr = 1e-1
opt.model_name = 'efficientnet_b6'
opt.batch_size = 10
opt.kfold = 5
## random seed
torch.manual_seed(42)
np.random.seed(42)

In [3]:
train_df = pd.read_csv('../train_new.csv', index_col=0)
train_df.head()
# train_df['label'].value_counts().plot()

,image,label
0,train_00000.png,1
1,train_00001.png,0
2,train_00002.png,0
3,train_00003.png,0
4,train_00004.png,0


## Train

In [4]:
## wandb
wandb.login()

run = wandb.init(
    project='HUAWEI_lane_rendering', 
    entity='qddse',
    name='efficientnet_b6_aug',
    reinit=True,
    config=opt,
    group='EffiNetb6',
    job_type='train_fold1'
)


wandb: Currently logged in as: qddse. Use `wandb login --relogin` to force relogin


In [5]:
import gc
gc.collect()


192

In [6]:
## Trainer
class Trainer:
    def __init__(self, dataloaders, optimizer, model, loss_fn, scheduler=None, device='cuda:0', amp=False, config=None):
        self.train_loader, self.val_loader = dataloaders
        self.train_loss = loss_fn
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.device = device
        self.amp = amp
        self.config = config
        self.model = model
        
    def train_one_epoch(self):
        self.model.cuda()
        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        running_loss = 0
        for bnum, data_cache in train_pbar:
            img = data_cache[0].cuda()
            label = data_cache[1].long().cuda()
            ## compute loss
            pred = self.model(img)
            loss = self.train_loss(pred, label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
        return running_loss / (len(self.train_loader))
    
    @torch.no_grad()
    def valid_one_epoch(self):
        self.model.eval()
        valid_pbar = tqdm(enumerate(self.val_loader), total=len(self.val_loader))
        valid_preds, valid_targets = [], []
        valid_acc = 0.
        for idx, cache in valid_pbar:
            img = cache[0].cuda()
            target = cache[1].cuda()
            pred = self.model(img)
            valid_acc += (pred.argmax(1) == target).sum().item()
            
        return valid_acc / len(self.val_loader.dataset)
            
    def train_one_epoch_with_mixup(self):
        '''
        Train one epoch using mixup
        '''
        if self.amp:
            scaler = GradScaler()
        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        running_loss = 0
        for bnum, data_cache in train_pbar:
            img = self._convert_if_not_tensor(data_cache[0], dtype=torch.float32)
            target = self._convert_if_not_tensor(data_cache[1], dtype=torch.float32)

            bs = img.shape[0]
            ## Support Amp with Mixup
            if self.amp:
                # Mixup
                if torch.randn(1)[0] < 0.5:
                    mix_img, tar_a, tar_b, lam = mixup_augmentation(img, target, alpha=0.5)
                    with autocast(enabled=True):
                        output = self.model(mix_img).squeeze()
                        ## mixup loss
                        loss_a = self.train_loss_fn(output, tar_a)
                        loss_b = self.train_loss_fn(output, tar_b)
                        loss = loss_a * lam + (1-lam) * loss_b
                        
                        loss = loss / self.config['N_ACCUM']
                    scaler.scale(loss).backward()
                    if (bnum + 1) % self.config['N_ACCUM'] == 0:
                        scaler.step(self.optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                        
                        if self.scheduler:
                            self. scheduler.step()
                    running_loss += (loss.item() * bs)
                    
                else:
                    with autocast(enabled=True):
                        output = self.model(img).squeeze()
                        loss = self.train_loss_fn(output, target)
                        loss = loss / self.config['N_ACCUM']
                    scaler.scale(loss).backward()
                    
                    if (bnum + 1) % self.config['N_ACCUM'] == 0:
                        scaler.step(self.optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                        
                        if self.scheduler:
                            self.scheduler.step()
                    running_loss += (loss.item() * bs)
                    
            else:  # no amp
                ## mixup allowed
                if torch.randn(1)[0] < 0.5:
                    mix_img, tar_a, tar_b, lam = mixup_augmentation(img, target, alpha=0.5)
                    output = self.model(mix_img).squeeze()
                    
                    ## loss
                    loss_a = self.train_loss_fn(output, tar_a)
                    loss_b = self.train_loss_fn(output, tar_b)
                    loss = loss_a * lam + (1-lam) * loss_b
                    loss = loss / self.config['N_ACCUM']
                    loss.backward()
                    
                    if (bnum + 1)% self.config['N_ACCUM'] == 0:
                        self.optimizer.step()
                        optimizer.zero_grad()
                        
                        if self.scheduler:
                            self.scheduler.step()
                    running_loss += (loss.item() * bs)
                else:
                    output = self.model(img).squeeze()
                    loss = self.train_loss_fn(output, target)
                    loss = loss / self.config['N_ACCUM']
                    loss.backward()

                    if (bnum + 1) % self.config['N_ACCUM'] == 0:
                        self.optimizer.step()
                        optimizer.zero_grad()

                        if self.scheduler:
                            self.scheduler.step()
                    running_loss += (loss.item() * bs)
            train_pbar.set_description(desc=f'loss: {loss.item():.4f}')
            running_loss /= len(self.train_loader)
            
            # Rescale the targets and output before chugging in a matrix
            output = output.sigmoid().detach() * 100.0
            target = target.detach() * 100.0
            train_preds += [output.cpu().numpy()]
            train_labels += [target.cpu().numpy()]
        
        all_train_preds = np.concatenate(train_preds)
        all_train_labels = np.concatenate(train_labels)
        
        # Tidy
        del output, target, train_preds, train_labels, loss, img, meta, all_train_preds, all_train_labels
        gc.collect()
        torch.cuda.empty_cache()
        
        return running_loss
    
    def fit(self, fold: int=None, epochs:int = 10, output_dir:str='./ckpts/826/04/', custom_name:str='model_best.pth', gain_train=None, is_wandb=True, patience=4):
        """
        Low-effort alternative for doing the complete training and validation process
        args:
            gain_train: str ,ckpts的地址
        """
        ## scheduler
        self.optimizer
        best_loss = int(1e7)
        best_acc = 0.0
        if gain_train:
            ckpt = torch.load(gain_train)
            model_ckpt = ckpt['model']
            self.model.load_state_dict(model_ckpt)
        for epoch in range(epochs):
            print(f"{'*'*20} Epoch: {epoch} / {epochs} {'*'*20}")
            
            train_running_loss = self.train_one_epoch()
            print(f"Training loss: {train_running_loss:.4f}")
            valid_acc = self.valid_one_epoch()
            print(f"Validation Acc : {valid_acc:.4f}")
            
            if train_running_loss < best_loss:
                best_loss = train_running_loss
                self.save_model(output_dir, custom_name, loss_final=best_loss)
                self.save_model(output_dir + '/fold_'+str(fold) + '/', 'model_' + str(epoch) + '.pth', best_loss)

                print(f"Saved model with train_loss: {best_loss:.4f}")
                es = 0
            else:
                es += 1
                print("Counter: {} / {}".format(es, patience))
                if es > patience:
                    print('Early Stop with train_loss: {}, val_acc:{}'.format(train_running_loss, valid_acc))
                    break
            if best_acc < valid_acc:
                best_acc = valid_acc
                self.save_model(output_dir + 'via_acc', custom_name)
        
            ## logger
            if is_wandb:
                run.log({'epoch:': epoch, 'Fold': fold, 'loss_train:': train_running_loss, 'val_acc': valid_acc})
            if self.scheduler:
                self.scheduler.step()
                
    def save_model(self, path, name, verbose=False, loss_final=None):
        """
        Saves the model at the provided destination
        """
        try:
            if not os.path.exists(path):
                os.mkdir(path)
        except:
            print("Errors encountered while making the output directory")
        torch.save({
            'model':self.model.state_dict(), 
            'loss': loss_final,
            },
            os.path.join(path, name)
        )
        if verbose:
            print(f"Model Saved at: {os.path.join(path, name)}")

In [7]:
def Convert_01(label):
    '''
    args:
        label:int
    '''
    if label == 0:
        return 0
    else:
        return 1
    
## 将label 转化为0-1
train_df['label_01'] = train_df['label'].apply(lambda x: Convert_01(x))
train_df.sample(20)

,image,label,label_01
15505,train_15505.png,0,0
9642,train_09642.png,0,0
23208,train_23208.png,0,0
7917,train_07917.png,0,0
23946,train_23946.png,0,0
13744,train_13744.png,0,0
6531,train_06531.png,0,0
16565,train_16565.png,0,0
23404,train_23404.png,0,0
250,train_00250.png,0,0


In [8]:
## new_Dataset
class Rane_Dataset(data.Dataset):
    '''
    args:
        imagedir: 存放image的地址
        df: Dataframe, train_df or val_df
        tranfrom: 
    '''
    def __init__(self, imagedir, df, transform, require_label=True):
        self.imagedir = imagedir
        self.df = df
        self.require_label = require_label
        if transform:
            self.transform = transform
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
    def __getitem__(self, idx):
        line = self.df.iloc[idx]
        ## img
        # image = Image.open(os.path.join(self.imagedir, line['image']))
        # image = image.convert('RGB')
        img = cv2.imread(os.path.join(self.imagedir, line['image']))
        img = img.astype(np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image = img)['image']  ## Notice!! 
        ## label
        label = line['label_01']
        # print(type(class_label))
        # if class_label == 0:
        #     label = 0
        # else:
        #     label = 1
        if self.require_label:
            # print(type(image))
            return (img, label)
        else:
            return img
        
    def __len__(self):
        return self.df.shape[0]
    
def build_loader_new(
        imagedir,
        batch_size,
        num_workers,
        metafile,
        require_label=True,
        transform = None,
):
    if transform == None:
        trfs = [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
        trfs = transforms.Compose(trfs)
    else:
        trfs = transform
        
    dataset = Rane_Dataset(imagedir, metafile, trfs, require_label)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )
    return dataloader

In [9]:
## train
from sklearn.model_selection import KFold

## modified model    
from models import Effnet, Swin_trm

# model = torchvision.models.resnet50(pretrained=False, num_classes=opt.num_classes).to(opt.device)
model = Effnet(model_name=opt.model_name)
optimizer = optim.AdamW(model.parameters(), lr=opt.lr, weight_decay=opt.weight_decay)
loss_fn = nn.CrossEntropyLoss()
# transform = T.Compose([
#                       T.Resize((224, 224)),
#                       #T.HorizontalFlip(p=0.5),
#                       T.RandomHorizontalFlip(0.5),
#                       T.RandomVerticalFlip(0.5),
#                       # A.VerticalFlip(p=0.5),
#                       # T.Random,
#                       # A.RandomContrast(p=0.5),
#                       T.RandomGrayscale(0.5),
#                       # A.RandomBrightness(p=0.5),
#                       T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#                       T.ToTensor(),
#                 ])
transform = A.Compose([
                      ## Swin-Resolution: (384,384) 
                      A.Resize(384, 384),
                      A.HorizontalFlip(p=0.5),
                      A.VerticalFlip(p=0.5),
                      A.RandomContrast(p=0.5),
                      A.RandomBrightness(p=0.5),
                      A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                ])
transform_val = A.Compose([
                      A.Resize(384, 384),
                      # A.HorizontalFlip(p=0.5),
                      # A.VerticalFlip(p=0.5),
                      # A.RandomContrast(p=0.5),
                      # A.RandomBrightness(p=0.5),
                      A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                ])
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= 20, eta_min=1e-4)

kf = KFold(n_splits=5)


No pretrained weights exist for this model. Using random initialization.


In [ ]:
for fold, split in enumerate(kf.split(train_df)):
    print(f"{'*'*20} Fold: {fold} / {opt.kfold} {'*'*20}")
    if fold == 0 :
        continue
    train_ = train_df.iloc[split[0]]
    val_ = train_df.iloc[split[1]]
    print(train_.shape[0], val_.shape[0])
    
    train_loader = build_loader_new(
        imagedir=opt.traindir,
        batch_size=opt.batch_size,
        num_workers=opt.num_workers,
        # metadir=opt.train_metadir,
        metafile=train_,
        require_label=True,
        transform = transform
    )
    val_loader = build_loader_new(
        imagedir=opt.traindir,
        batch_size=opt.batch_size,
        num_workers=opt.num_workers,
        # metadir=opt.train_metadir,
        metafile=val_,
        require_label=True,
        transform=transform_val
    )
    Train = Trainer((train_loader, val_loader), optimizer, model, loss_fn, scheduler=lr_scheduler)
    Train.fit(fold=fold, epochs=opt.epochs, output_dir='../ckpts/9_04/', gain_train='../ckpts/9_04/model_best.pth')
    del Train
    gc.collect()

******************** Fold: 0 / 5 ********************
******************** Fold: 1 / 5 ********************
23331 5833
******************** Epoch: 0 / 5 ********************


  0%|          | 0/2334 [00:00<?, ?it/s]

Training loss: 2.7740


  0%|          | 0/584 [00:00<?, ?it/s]

Validation Acc : 0.8843
Model Saved at: ../ckpts/9_04//fold_1/model_0.pth
Saved model with train_loss: 2.7740
******************** Epoch: 1 / 5 ********************


...

  0%|          | 0/2334 [00:00<?, ?it/s]

Training loss: 0.7093


  0%|          | 0/584 [00:00<?, ?it/s]

Validation Acc : 0.8865
Model Saved at: ../ckpts/9_04//fold_2/model_0.pth
Saved model with train_loss: 0.7093
******************** Epoch: 1 / 5 ********************


  0%|          | 0/2334 [00:00<?, ?it/s]

In [ ]:
os.system('shutdown')